In [1]:
import pandas as pd

years = range(2014, 2026)
files = [f"{year}world.csv" for year in years]

dataframes = []
for file in files:
    df = pd.read_csv(file)
    df['time'] = pd.to_datetime(df['time'], utc=True)
    dataframes.append(df)

In [2]:
merged_df = pd.concat(dataframes, ignore_index=True)

# Sort by time in descending order (newest to oldest)
merged_df.sort_values(by='time', ascending=False, inplace=True)

In [3]:
print(df.head())

                              time  latitude  longitude    depth  mag magType  \
0 2025-03-31 15:58:26.585000+00:00  -23.7295   -66.7675  202.242  4.5      mb   
1 2025-03-31 15:54:19.308000+00:00  -10.0526   119.0351   28.472  5.6     mww   
2 2025-03-31 15:06:01.638000+00:00  -52.2187    27.9676   10.000  4.7      mb   
3 2025-03-31 13:36:16.254000+00:00    0.6527   -29.6700   10.000  5.2      mb   
4 2025-03-31 13:20:45.875000+00:00  -20.4567  -173.7708   25.343  4.9      mb   

     nst    gap    dmin   rms  ...                   updated  \
0   46.0   93.0   1.511  1.13  ...  2025-03-31T16:22:19.040Z   
1  108.0   25.0   1.301  0.92  ...  2025-03-31T16:39:23.416Z   
2   19.0   91.0  20.499  0.49  ...  2025-03-31T15:39:47.040Z   
3   61.0   89.0   8.952  1.01  ...  2025-03-31T14:49:28.040Z   
4   50.0  125.0   3.871  1.11  ...  2025-03-31T14:26:23.040Z   

                                              place        type  \
0  70 km NW of San Antonio de los Cobres, Argentina  earthqua

In [4]:
# Convert the 'time' column back to ISO8601 format with milliseconds and a trailing "Z"
# This step assumes that you want exactly three decimal places for milliseconds.
merged_df['time'] = merged_df['time'].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
merged_df['time'] = merged_df['time'].str[:-3] + "Z"

In [5]:
print(df.head())

                              time  latitude  longitude    depth  mag magType  \
0 2025-03-31 15:58:26.585000+00:00  -23.7295   -66.7675  202.242  4.5      mb   
1 2025-03-31 15:54:19.308000+00:00  -10.0526   119.0351   28.472  5.6     mww   
2 2025-03-31 15:06:01.638000+00:00  -52.2187    27.9676   10.000  4.7      mb   
3 2025-03-31 13:36:16.254000+00:00    0.6527   -29.6700   10.000  5.2      mb   
4 2025-03-31 13:20:45.875000+00:00  -20.4567  -173.7708   25.343  4.9      mb   

     nst    gap    dmin   rms  ...                   updated  \
0   46.0   93.0   1.511  1.13  ...  2025-03-31T16:22:19.040Z   
1  108.0   25.0   1.301  0.92  ...  2025-03-31T16:39:23.416Z   
2   19.0   91.0  20.499  0.49  ...  2025-03-31T15:39:47.040Z   
3   61.0   89.0   8.952  1.01  ...  2025-03-31T14:49:28.040Z   
4   50.0  125.0   3.871  1.11  ...  2025-03-31T14:26:23.040Z   

                                              place        type  \
0  70 km NW of San Antonio de los Cobres, Argentina  earthqua

In [6]:
# Save the merged DataFrame to a new CSV file
merged_df.to_csv("2014-2025earthquakes.csv", index=False)

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Point
!pip install shapely
!pip install geopandas

In [ ]:
# prompt: i want to import 2024-2025.csv as a panda frame

# Assuming the CSV file is in the current working directory.
# If it's in a different location, provide the full path.
df = pd.read_csv('2014-2025earthquakes.csv')
print(df.head())  # Display the first few rows of the DataFrame



In [ ]:
def get_continent_from_lat_lon(lat, lon):
    if -56 <= lat <= 83:  # Approximate range of inhabited continents
        if -170 <= lon <= -50:
            return "NA"  # North America
        elif -50 <= lon <= 60:
            return "EU" if lat >= 35 else "AF"  # Europe or Africa
        elif 60 <= lon <= 150:
            return "AS"  # Asia
        elif 150 <= lon or lon <= -170:
            return "OC"  # Oceania
        elif -80 <= lon <= -30 and -60 <= lat <= 0:
            return "SA"  # South America
    return "AC"

In [ ]:
# Load tectonic plate shapefile (Download from USGS or GPlates)
plates = gpd.read_file("PB2002_plates.json")
def get_plate(lat, lon, gdf=plates):
    """Returns the tectonic plate for a given lat/lon."""
    point = Point(lon, lat)
    match = gdf[gdf.geometry.contains(point)]
    return match['PlateName'].values[0] if not match.empty else "Unknown"

In [ ]:
try:
    df = pd.read_csv('2024-2025earthquakes.csv')

    # Extract location from the 'Country' column
    df['Country'] = df['place'].str.split(',').str[-1].str.strip()
    for index, row in df.iterrows():
        df.at[index, 'Continent'] = get_continent_from_lat_lon(row['latitude'], row['longitude'])
        # df.at[index, 'place'] = get_continent_from_lat_lon(row['latitude'], row['longitude'])
        df.at[index, 'Plate'] = get_plate(row['latitude'], row['longitude'])
    print(df.head(5))

    # Export the updated DataFrame to a new CSV file
    df.to_csv('updated_2024-2025.csv', index=False)  # Set index=False to avoid writing row indices

    print("New CSV file 'updated_2024-2025.csv' created successfully.")

except FileNotFoundError:
    print("Error: '2024-2025.csv' not found. Please make sure the file exists in the current directory or provide the correct path.")
except KeyError:
    print("Error: 'Country' column not found in the CSV file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")